In [120]:
import pandas as pd
import numpy as np

In [121]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# **DATASET LOADING**

In [123]:
df = pd.read_csv("../data/processed/01_Data_Cleaning.csv")
df['date'] = pd.to_datetime(df['date'])

df.head()

,date,sell_quantity,sell_value,buy_quantity,buy_value,daily_balance,inventory,avg_temperature,precipitation,incc,is_holiday
0,2024-01-01,NaN,NaN,NaN,NaN,0,175,24.9,0.1,0.0027,1
1,2024-01-02,163.0,32.10,280.0,28.97,117,292,25.0,0.2,0.0027,0
2,2024-01-03,215.0,31.13,NaN,28.97,-215,77,24.8,4.9,0.0027,0
3,2024-01-04,NaN,NaN,240.0,28.43,240,317,24.8,0.0,0.0027,0
4,2024-01-05,55.0,35.50,240.0,28.43,185,502,25.1,0.0,0.0027,0


# **FEATURE ENGINEERING**

## **DEMAND, SUPPLY AND STOCK DYNAMICS**

<br>
Gross margin per unit (selling price / buying price) to measure profitability. It will be the primary target variable for profit optimization.

In [139]:
df['price_margin'] = df.apply(lambda row: (row['sell_value'] / row['buy_value']) if row['sell_value'] != 0 else np.nan, axis = 1)

3-day rolling average of the daily selling and buying prices, used to smooth out short-term fluctuations and highlight trends.  
The 3-day standard deviation captures short-term volatility in selling prices.

In [250]:
df['rolling_mean_sell_value'] = df['sell_value'].rolling(window = 3, min_periods = 1).mean()
df['rolling_mean_buy_value'] = df['buy_value'].rolling(window = 3, min_periods = 1).mean()

df['rolling_std_sell_value_3d'] = df['sell_value'].rolling(window = 3).std()

<br>
Lag features of daily quantity sold (1-day and 7-day), allowing the model to detect temporal patterns or autocorrelation in demand. As well as, previous day’s inventory value, to help understand dynamic stock variation, and 7-day rolling average of inventory, reflecting smoothed stock availability.
<br>
The standard deviation helps identify sale volatility over the past 7 days.

In [129]:
df['sell_quantity_lag_1'] = df['sell_quantity'].shift(1)
df['sell_quantity_lag_7'] = df['sell_quantity'].shift(7)
df['sell_quantity_std_7d'] = df['sell_quantity'].rolling(window = 7).std()

df['inventory_lag_1'] = df['inventory'].shift(1)
df['avg_inventory_rolling'] = df['inventory'].rolling(window = 7).mean()

<br>
To measure how quickly inventory is being sold relative to the previous day's stock level.

In [246]:
df['inventory_turnover'] = df['sell_quantity'] / (df['inventory_lag_1'] + 1e-6)

<br>
7-day average of quantity sold — helpful to track recent sales momentum.

In [131]:
df['avg_sell_quantity_last_7d'] = df['sell_quantity'].rolling(window = 7).mean()

<br>
Cumulative sum of product units sold, indicating total sales volume evolution over time.

In [135]:
df['cumulative_sell_quantity'] = df['sell_quantity'].cumsum()

<br>
Daily percentage change in selling and buying prices, indicating price volatility.

In [137]:
df['sell_value_pct_change'] = df['sell_value'].pct_change().replace([np.inf, -np.inf], np.nan)
df['buy_value_pct_change'] = df['buy_value'].pct_change().replace([np.inf, -np.inf], np.nan)

## **EXOGENOUS VARIABLES: WEATHER AND HOLIDAYS**

Flags for moderate (2.5–10 mm) and heavy (≥10 mm) daily rainfall, based on Brazilian climate thresholds, which may affect sales of construction materials like cement.

In [142]:
df['is_moderate_rain'] = (df['precipitation'] >= 2.5) & (df['precipitation'] < 10)
df['is_heavy_rain'] = df['precipitation'] >= 10

<br>
Temperature bins that categorize daily average temperature into three ranges based on cement curing guidelines:

- **low**: below 10 °C. May slow down or impair proper curing; 
- **optimal**: between 10 °C and 25 °C. Ideal range for cement setting and strength development;
- **high**: above 25 °C. May accelerate drying and reduce surface quality.

In [255]:
df['temperature_bin'] = pd.cut(
    df['avg_temperature'],
    bins=[-float('inf'), 10, 25, float('inf')],
    labels=['low', 'optimal', 'high']
)

<br>
Date-based time features that help the model capture seasonal patterns in demand and pricing.

In [146]:
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week
df['quarter'] = df['date'].dt.quarter

<br>
Incorporate anticipation effect.

In [237]:
k = 1
df.loc[0, 'days_until_holiday'] = 0

for i in range(1, len(df)):
    if (df.loc[i, 'is_holiday'] == 1):
        h = df.loc[i, 'date']

        for j in range(k, i):
            df.loc[j, 'days_until_holiday'] = abs((h - df.loc[j, 'date']).days)

        k = i + 1

df['days_until_holiday'] = df['days_until_holiday'].fillna(0)

In [239]:
df[['date', 'is_holiday', 'days_until_holiday']].head(125)

,date,is_holiday,days_until_holiday
0,2024-01-01,1,0
1,2024-01-02,0,87
2,2024-01-03,0,86
3,2024-01-04,0,85
4,2024-01-05,0,84
...,...,...,...
120,2024-04-30,0,1
121,2024-05-01,1,0
122,2024-05-02,0,68
123,2024-05-03,0,67


In [207]:
df[df['is_holiday'] == 1][['date', 'is_holiday']].head(30)

,date,is_holiday
0,2024-01-01,1
88,2024-03-29,1
111,2024-04-21,1
121,2024-05-01,1
190,2024-07-09,1
213,2024-08-01,1
250,2024-09-07,1
285,2024-10-12,1
306,2024-11-02,1
319,2024-11-15,1


## **INFLATION-AJUSTED FEATURES**

Lagged values of monthly INCC inflation rate, allowing the model to capture inflation trends and their delayed effects on pricing.

In [151]:
df['incc_lag_month'] = df['incc'].shift(30)

<br>
Month-over-month change in the INCC inflation rate.

In [153]:
df['incc_monthly_change'] = df['incc'] - df['incc'].shift(30)

<br>

The `sell_value_pct_vs_incc` feature represents the difference between the daily percentage change in selling price and the INCC inflation rate.
  - A positive value means the selling price increased more than inflation on that day, indicating a potential real price gain;
  - A negative value means the price increased less than inflation, or even decreased, indicating a real loss in value;
  - A value close to zero means the price kept pace with inflation, a neutral adjustment.

In [155]:
df['sell_value_pct_vs_incc'] = df['sell_value_pct_change'] - df['incc']

In [182]:
df.to_csv("../data/processed/02_EDA.csv", index = False)

In [184]:
df

,date,sell_quantity,sell_value,buy_quantity,buy_value,daily_balance,inventory,avg_temperature,precipitation,incc,...,is_heavy_rain,temperature_bin,month,week,quarter,is_day_before_holiday,is_day_after_holiday,incc_lag_month,incc_monthly_change,sell_value_pct_vs_incc
0,2024-01-01,NaN,NaN,NaN,NaN,0,175,24.9,0.1,0.0027,...,False,medium,1,1,1,0,0,NaN,NaN,NaN
1,2024-01-02,163.0,32.10,280.0,28.97,117,292,25.0,0.2,0.0027,...,False,medium,1,1,1,0,1,NaN,NaN,NaN
2,2024-01-03,215.0,31.13,NaN,28.97,-215,77,24.8,4.9,0.0027,...,False,medium,1,1,1,0,0,NaN,NaN,-0.032918
3,2024-01-04,NaN,NaN,240.0,28.43,240,317,24.8,0.0,0.0027,...,False,medium,1,1,1,0,0,NaN,NaN,-0.002700
4,2024-01-05,55.0,35.50,240.0,28.43,185,502,25.1,0.0,0.0027,...,False,medium,1,1,1,0,0,NaN,NaN,0.137679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,2025-04-26,NaN,NaN,NaN,30.22,0,475,21.6,1.6,0.0052,...,False,medium,4,17,2,0,0,0.0039,0.0013,-0.005200
482,2025-04-27,NaN,NaN,NaN,30.22,0,475,22.3,0.1,0.0052,...,False,medium,4,17,2,0,0,0.0039,0.0013,-0.005200
483,2025-04-28,39.0,35.17,NaN,30.22,-39,436,21.7,7.6,0.0052,...,False,medium,4,18,2,0,0,0.0039,0.0013,0.001959
484,2025-04-29,107.0,34.71,NaN,30.22,-107,329,21.0,0.3,0.0052,...,False,medium,4,18,2,0,0,0.0039,0.0013,-0.018279
